In [1]:
from tmilib import *

In [2]:
from reconstruct_focus_times_common import *
from reconstruct_focus_times import ReconstructFocusTimesBaseline
from session_tracker import get_focused_tab

In [3]:
'''
for user in list_users():
  ordered_visits = get_history_ordered_visits_for_user(user)
  if len(ordered_visits) == 0:
    continue
  last_visit = ordered_visits[-1]
  last_visit_time = float(last_visit['visitTime']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)
'''

"\nfor user in list_users():\n  ordered_visits = get_history_ordered_visits_for_user(user)\n  if len(ordered_visits) == 0:\n    continue\n  last_visit = ordered_visits[-1]\n  last_visit_time = float(last_visit['visitTime']) / 1000.0\n  print user, datetime.datetime.fromtimestamp(last_visit_time)\n"

In [4]:
'''
for user in list_users():
  tab_focus_times = get_tab_focus_times_for_user(user)
  if len(tab_focus_times) == 0:
    continue
  last_visit = tab_focus_times[-1]
  last_visit_time = float(last_visit['end']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)
'''

"\nfor user in list_users():\n  tab_focus_times = get_tab_focus_times_for_user(user)\n  if len(tab_focus_times) == 0:\n    continue\n  last_visit = tab_focus_times[-1]\n  last_visit_time = float(last_visit['end']) / 1000.0\n  print user, datetime.datetime.fromtimestamp(last_visit_time)\n"

In [5]:
def get_evaluated_reference_and_reconstructed_focus_times_for_user(user):
  tab_focus_times = get_tab_focus_times_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return [],[]
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  ref_start_time = max(ref_start_time, 1458371950000) # march 18th. may have had some data loss prior to that
  ref_end_time = max(ref_end_time, 1458371950000)
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  return evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times

def get_evaluation_stats_for_user(user):
  evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
    return {}
  return evalutate_tab_focus_reconstruction(evaluated_tab_focus_times, evaluated_reconstructed_tab_focus_times)

'''
def get_evaluation_stats_for_user(user):
  #evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  #if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
  #  return {}
  tab_focus_times = get_tab_focus_times_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return {}
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  result_reference = second_to_active_url(evaluated_tab_focus_times, ref_start_time, ref_end_time)
  result_reconstructed = second_to_active_url(evaluated_reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  return evaluation_stats_for_reconstruction(result_reference, result_reconstructed)
  #print tab_focus_times[0].keys()
  #print list(reconstructed_tab_focus_times)[0].keys()
'''

'\ndef get_evaluation_stats_for_user(user):\n  #evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)\n  #if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:\n  #  return {}\n  tab_focus_times = get_tab_focus_times_for_user(user)\n  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))\n  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:\n    return {}\n  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))\n  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))\n  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)\n  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_

In [6]:
class ClosestVisitTimeForUrl:
  def __init__(self, tab_focus_times):
    self.visits = tab_focus_times
    self.url_to_visit_times = {} # url to start of visit
    self.visit_time_to_visit = {}
    for visit in self.visits: # these visits are time-ordered
      start = visit['start']
      url = visit['url']
      if url not in self.url_to_visit_times:
        self.url_to_visit_times[url] = []
      self.url_to_visit_times[url].append(start)
      self.visit_time_to_visit[start] = visit
  def get_visit_from_time(self, target_visit_time):
    self.visit_time_to_visit.get(target_visit_time, None)
  def find_closest_visit(self, url, target_visit_time):
    if url not in self.url_to_visit_times:
      return None
    all_visits = self.url_to_visit_times[url]
    closest_visit = None
    for visit in all_visits:
      if closest_visit == None:
        closest_visit = visit
        continue
      if abs(visit - target_visit_time) < abs(closest_visit - target_visit_time):
        closest_visit = visit
    return closest_visit

In [7]:
def get_span_alignment_stats_for_user(user):
  evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
    return {}
  closest_visit_times_for_reference = ClosestVisitTimeForUrl(evaluated_tab_focus_times)
  closest_visit_times_for_reconstructed = ClosestVisitTimeForUrl(evaluated_reconstructed_tab_focus_times)
  stats = Counter()
  domains_where_reconstructed_has_no_reference = Counter()
  domains_where_reference_has_no_reconstructed = Counter()
  urls_where_reconstructed_has_no_reference = Counter()
  urls_where_reference_has_no_reconstructed = Counter()
  reconstructed_transition_types_with_no_corresponding_reference = Counter()
  reconstructed_transition_types_total = Counter()
  reconstructed_visits_labeled = []
  reference_visits_labeled = []
  for visit in evaluated_reconstructed_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    closest_time = closest_visit_times_for_reference.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if 'transition' in visit:
      transition = visit['transition']
      reconstructed_transition_types_total[transition] += 1
    if closest_time == None or abs(start_time - closest_time) > 5*1000: # more than 5 seconds off
      stats['reconstructed_has_no_corresponding_reference'] += 1
      domains_where_reconstructed_has_no_reference[domain] += 1
      urls_where_reconstructed_has_no_reference[url] += 1
      visit_labeled['have_reference'] = False
      if 'transition' in visit:
        transition = visit['transition']
        reconstructed_transition_types_with_no_corresponding_reference[transition] += 1
    else:
      stats['reconstructed_has_corresponding_reference'] += 1
      visit_labeled['have_reference'] = True
    reconstructed_visits_labeled.append(visit_labeled)
  for visit in evaluated_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    closest_time = closest_visit_times_for_reconstructed.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if closest_time == None or abs(start_time - closest_time) > 5*1000: # more than 5 seconds off
      stats['reference_has_no_corresponding_reconstructed'] += 1
      domains_where_reference_has_no_reconstructed[domain] += 1
      urls_where_reference_has_no_reconstructed[url] += 1
      visit_labeled['have_reconstructed'] = False
    else:
      stats['reference_has_corresponding_reconstructed'] += 1
      visit_labeled['have_reconstructed'] = True
    reference_visits_labeled.append(visit_labeled)
  return {
    'stats': stats,
    'domains_where_reconstructed_has_no_reference': domains_where_reconstructed_has_no_reference,
    'domains_where_reference_has_no_reconstructed': domains_where_reference_has_no_reconstructed,
    'urls_where_reconstructed_has_no_reference': urls_where_reconstructed_has_no_reference,
    'urls_where_reference_has_no_reconstructed': urls_where_reference_has_no_reconstructed,
    'reconstructed_visits_labeled': reconstructed_visits_labeled,
    'reference_visits_labeled': reference_visits_labeled,
    'reconstructed_transition_types_with_no_corresponding_reference': reconstructed_transition_types_with_no_corresponding_reference,
    'reconstructed_transition_types_total': reconstructed_transition_types_total,
  }

In [8]:
user_to_span_alignment_data = {}

for user in list_users():
  data = get_span_alignment_stats_for_user(user)
  if 'stats' not in data:
    continue
  user_to_span_alignment_data[user] = data


In [9]:
overall_span_alignment_stats = Counter()
for user,data in user_to_span_alignment_data.items():
  stats = data['stats']
  for k,v in stats.items():
    overall_span_alignment_stats[k] += v

print_counter(overall_span_alignment_stats)
# 60% of reference have corresponding reconstructed
# 84% of reconstructed have corresponding reference

reference_has_no_corresponding_reconstructed 1031863
reference_has_corresponding_reconstructed 760631
reconstructed_has_corresponding_reference 654689
reconstructed_has_no_corresponding_reference 116359


In [10]:
overall_transitions_with_no_reference = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['reconstructed_transition_types_with_no_corresponding_reference'].items():
    overall_transitions_with_no_reference[k] += v

print_counter(overall_transitions_with_no_reference)

link 86201
form_submit 13390
reload 7529
typed 3852
generated 2530
auto_bookmark 1954
auto_toplevel 860
keyword 38
keyword_generated 5


In [11]:
overall_transitions_total_in_evaluated_reconstruction = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['reconstructed_transition_types_total'].items():
    overall_transitions_total_in_evaluated_reconstruction[k] += v

print_counter(overall_transitions_total_in_evaluated_reconstruction)


link 594180
form_submit 94643
reload 28534
typed 20825
auto_bookmark 15918
generated 13574
auto_toplevel 3112
keyword 256
keyword_generated 6


In [12]:
# history transition types that have highest levels of no-corresponding-reference
for k,v in overall_transitions_total_in_evaluated_reconstruction.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)

form_submit 0.14147903173
auto_bookmark 0.122754114839
keyword 0.1484375
typed 0.184969987995
auto_toplevel 0.276349614396
reload 0.263860657461
generated 0.186385737439
link 0.145075566327
keyword_generated 0.833333333333


In [13]:
overall_transitions_total_in_reconstruction = Counter()

for user in list_users_with_hist():
  for visit in get_reconstruct_focus_times_baseline_for_user(user):
    transition = visit['transition']
    overall_transitions_total_in_reconstruction[transition] += 1

print_counter(overall_transitions_total_in_reconstruction)

link 4996461
form_submit 767306
reload 269928
typed 256299
auto_bookmark 135748
generated 131291
auto_toplevel 99731
keyword 2170
keyword_generated 20


In [14]:
overall_transitions_total = Counter()

for user in list_users_with_hist():
  for visit in get_history_ordered_visits_for_user(user):
    transition = visit['transition']
    overall_transitions_total[transition] += 1

print_counter(overall_transitions_total)

link 6157728
form_submit 1099071
reload 412746
typed 265271
auto_bookmark 140369
manual_subframe 137829
generated 131656
auto_toplevel 100801
keyword 2173
auto_subframe 302
keyword_generated 21


In [15]:
for k,v in overall_transitions_total.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)

form_submit 0.0121830163838
auto_subframe 0.0
auto_bookmark 0.0139204525216
keyword 0.0174873446848
typed 0.01452099928
auto_toplevel 0.00853166139225
reload 0.0182412427982
generated 0.0192167466731
link 0.0139988320368
manual_subframe 0.0
keyword_generated 0.238095238095


In [16]:
for k,v in overall_transitions_total_in_reconstruction.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)



form_submit 0.0174506650541
auto_bookmark 0.014394318885
keyword 0.0175115207373
typed 0.0150293212225
auto_toplevel 0.00862319639831
reload 0.027892623218
generated 0.0192701708419
link 0.0172524112567
keyword_generated 0.25


In [17]:
optimal_reconstruction_stats_spans = Counter()
optimal_reconstruction_stats_times = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reconstructed_visits_labeled']:
    if visit['have_reference']:
      optimal_reconstruction_stats_spans['reconstructed_has_reference'] += 1
    else:
      optimal_reconstruction_stats_spans['reconstructed_has_no_reference'] += 1
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    if visit['have_reconstructed']:
      optimal_reconstruction_stats_spans['reference_has_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_can_be_reconstructed'] += length
    else:
      optimal_reconstruction_stats_spans['reference_has_no_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_cannot_be_reconstructed'] += length

In [18]:
print_counter(optimal_reconstruction_stats_spans)
# reference has reconstructed 60% = 80332.0 / (80332 + 54018)
# reconstructed has reference 84% = 55748.0 / (10948 + 55748)

reference_has_no_reconstructed 1031863
reference_has_reconstructed 760631
reconstructed_has_reference 654689
reconstructed_has_no_reference 116359


In [19]:
print_counter(optimal_reconstruction_stats_times)
# seconds of reference that can be reconstructed 57% = 1254885375.005127 / (1254885375.005127 + 947693694.9926758)

seconds_of_reference_that_cannot_be_reconstructed 16716741805.9
seconds_of_reference_that_can_be_reconstructed 11083055845.1


In [20]:
domain_to_seconds_that_cannot_be_reconstructed = Counter()
url_to_seconds_that_cannot_be_reconstructed = Counter()
domain_to_spans_that_cannot_be_reconstructed = Counter()
url_to_spans_that_cannot_be_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    if not visit['have_reconstructed']:
      domain_to_seconds_that_cannot_be_reconstructed[domain] += length
      domain_to_spans_that_cannot_be_reconstructed[domain] += 1
      url_to_seconds_that_cannot_be_reconstructed[url] += length
      url_to_spans_that_cannot_be_reconstructed[url] += 1

In [21]:
print_counter(domain_to_spans_that_cannot_be_reconstructed)

www.mturk.com 313408
www.facebook.com 43402
www.youtube.com 37278
newtab 35322
www.google.com 35245
mail.google.com 34260
www.reddit.com 23544
docs.google.com 14622
hitgrabber.net 12887
www.netflix.com 12110
www.mturkgrind.com 8823
www.twitch.tv 8798
www.amazon.com 8265
apps.facebook.com 7019
forums.somethingawful.com 6904
totalviewschool.k12.com 6444
www.swagbucks.com 5418
us-mg5.mail.yahoo.com 5130
 4792
www.tumblr.com 4387
game.soul-arena.com 4115
mturkforum.com 4028
www.mturkcrowd.com 3503
calendar.google.com 3357
play.pokemonshowdown.com 3172
us20.chatzy.com 3026
www.hulu.com 2862
en.wikipedia.org 2514
drive.google.com 2496
turkopticon.ucsd.edu 2427
www.google.co.in 2310
www.ebay.com 2183
www.csgodouble.com 2100
www.surveymonkey.com 2095
turkernation.com 2095
twitter.com 2053
mail.aol.com 1916
www.linkedin.com 1868
perk.tv 1808
thewatchseries.to 1761
payments.amazon.com 1729
klbibkeccnjlkjkiokjodocebajanakg 1701
ma.brazzers.com 1591
hw.reddit.com 1552
www.imdb.com 1491
us-mg6.mail

In [22]:
print_counter(domain_to_seconds_that_cannot_be_reconstructed)

www.mturk.com 3106209289.03
www.facebook.com 1164556817.99
www.youtube.com 963976398.991
www.reddit.com 553651184.972
mail.google.com 514187443.003
www.netflix.com 421403701.995
apps.facebook.com 321295679.005
docs.google.com 289080081.015
www.google.com 236035046.992
newtab 159834260.971
www.twitch.tv 153229513.012
forums.somethingawful.com 142865792.997
www.amazon.com 128976552.991
www.tumblr.com 124531237.994
us-mg5.mail.yahoo.com 106979334.007
www.mturkgrind.com 98730906.0117
hitgrabber.net 84484867.0034
www.surveymonkey.com 82741488.9963
quarks.inr.ac.ru 75533467.9988
forums.animez.to 68253467.0022
upenn.co1.qualtrics.com 66567017.999
tweetdeck.twitter.com 61957067.9968
plinga.com 60023886.9971
perk.tv 58273933.0037
play.pokemonshowdown.com 56107871.9946
www.lipstickalley.com 53819528.9968
mturkforum.com 52867379.0029
turkernation.com 51297369.012
allmyvideos.net 50222240.0012
soundcloud.com 49891551.9998
imgur.com 48943660.9961
yalesurvey.qualtrics.com 46414995.9954
templates.tex

In [23]:
print_counter(url_to_spans_that_cannot_be_reconstructed)

https://www.mturk.com/mturk/dashboard 55433
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 38207
chrome://newtab/ 35322
https://www.facebook.com/ 24001
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 19857
https://mail.google.com/mail/u/0/#inbox 14013
http://hitgrabber.net/ 12887
https://www.google.com/_/chrome/newtab?espv=2&ie=UTF-8 10425
https://www.mturk.com/mturk/myhits 6386
http://game.soul-arena.com/ 4115
 3334
https://www.mturk.com/HM 3039
https://us20.chatzy.com/11920092594925 3026
https://www.mturk.com/mturk/findhits?match=true 2931
https://www.mturk.com/mturk/submit 2715
https://mail.google.com/mail/u/1/#inbox 2620
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 2472
https://www.tumblr.com/dashboard 2293
https://www.mturk.com/mturk/findhits?match=false 1933
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 1902
https://www.mturk.com/mturk/findhits?match=false&hit_scraper# 1766
https://mail.google.com/mail/u/0/

In [24]:
print_counter(url_to_seconds_that_cannot_be_reconstructed)

https://www.mturk.com/mturk/dashboard 819853953.992
https://www.facebook.com/ 742395830.016
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 341962508.004
chrome://newtab/ 159834260.971
https://mail.google.com/mail/u/0/#inbox 158852682.007
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 152043276.999
https://www.mturk.com/mturk/findhits?match=true 113784877.001
http://hitgrabber.net/ 84484867.0034
https://www.tumblr.com/dashboard 79354927.9956
https://docs.google.com/spreadsheets/d/1ZBzbhT5itY-L75wfjrNVJQt177kWhXh72bCTqCfMQyE/edit#gid=2096979058 75842983.0
http://quarks.inr.ac.ru/2004/proceedings/Quarks/kuraev.pdf 75533467.9988
https://mail.google.com/mail/#inbox 68414218.9988
https://tweetdeck.twitter.com/# 58484884.999
https://www.mturk.com/mturk/myhits 57807538.019
https://www.google.com/_/chrome/newtab?espv=2&ie=UTF-8 56290382.0071
https://mail.google.com/mail/u/1/#inbox 53784284.0078
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 53470911.0

In [25]:
overall_domains_where_reconstructed_has_no_reference = Counter()
overall_domains_where_reference_has_no_reconstructed = Counter()
overall_urls_where_reconstructed_has_no_reference = Counter()
overall_urls_where_reference_has_no_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['domains_where_reconstructed_has_no_reference'].items():
    overall_domains_where_reconstructed_has_no_reference[k] += v
  for k,v in data['domains_where_reference_has_no_reconstructed'].items():
    overall_domains_where_reference_has_no_reconstructed[k] += v
  for k,v in data['urls_where_reconstructed_has_no_reference'].items():
    overall_urls_where_reconstructed_has_no_reference[k] += v
  for k,v in data['urls_where_reference_has_no_reconstructed'].items():
    overall_urls_where_reference_has_no_reconstructed[k] += v


In [26]:
print_counter(overall_urls_where_reconstructed_has_no_reference)

https://www.mturk.com/mturk/dashboard 2554
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 1890
https://www.mturk.com/mturk/checkregistration 1580
https://www.facebook.com/ 1457
https://www.mturk.com/mturk/previewandaccept?groupId=35DGDVUOHKKVOAMD7VZRLQEQ0J7YPA 1209
https://www.mturk.com/mturk/previewandaccept?groupId=3NYGPW9OUZ9VT0WI3H1AZXNS6BEZP1 1199
https://www.mturk.com/mturk/previewandaccept?groupId=3SFJXQ9EG58Y3WRLXCW2QCVUYYDXDX 851
https://www.mturk.com/mturk/previewandaccept?groupId=3PBTVBPQ9UI5R0A3PW7WXNE8W4CLGC 842
https://www.mturk.com/mturk/sortmyhits?searchSpec=HITSearch%23T%231%2325%23-1%23T%23%21Status%210%21rO0ABXQACEFzc2lnbmVk%21%23%21Deadline%211%21%23%21 670
https://perk.tv/video/6268/perk-app-trailers 553
https://www.mturk.com/mturk/endsignin 475
https://mail.google.com/mail/u/0/#inbox 464
https://www.mturk.com/mturk/findhits?match=false 389
https://www.youtube.com/ 385
https://perk.tv/video/248/perk-app-trailers 381
https://www.mturk.com

In [27]:
print_counter(overall_urls_where_reference_has_no_reconstructed)

https://www.mturk.com/mturk/dashboard 55433
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 38207
chrome://newtab/ 35322
https://www.facebook.com/ 24001
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 19857
https://mail.google.com/mail/u/0/#inbox 14013
http://hitgrabber.net/ 12887
https://www.google.com/_/chrome/newtab?espv=2&ie=UTF-8 10425
https://www.mturk.com/mturk/myhits 6386
http://game.soul-arena.com/ 4115
 3334
https://www.mturk.com/HM 3039
https://us20.chatzy.com/11920092594925 3026
https://www.mturk.com/mturk/findhits?match=true 2931
https://www.mturk.com/mturk/submit 2715
https://mail.google.com/mail/u/1/#inbox 2620
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 2472
https://www.tumblr.com/dashboard 2293
https://www.mturk.com/mturk/findhits?match=false 1933
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 1902
https://www.mturk.com/mturk/findhits?match=false&hit_scraper# 1766
https://mail.google.com/mail/u/0/

In [28]:
print_counter(overall_domains_where_reconstructed_has_no_reference)

www.mturk.com 27845
www.google.com 7131
www.youtube.com 6835
www.facebook.com 3823
docs.google.com 2943
i.imgur.com 2884
www.reddit.com 2322
perk.tv 1972
imgur.com 1868
mail.google.com 1363
www.mturkgrind.com 1271
earnhoney.com 1252
www.amazon.com 1233
i.ytimg.com 1011
www.host2inger.com 900
www.surveymonkey.com 840
work.crowdsurfwork.com 794
www.tohosting.tk 782
vitals-2.crowdcomputingsystems.com 763
mturkforum.com 726
local.cs-int-592.com:5550 640
www.netflix.com 534
turkernation.com 455
www.pinterest.com 432
mail.aol.com 428
www.hulu.com 411
turkopticon.ucsd.edu 407
personalityfactors.net 396
www.ebay.com 395
www.buzzfeed.com 375
www.google.co.in 367
twitter.com 340
thewatchseries.to 336
www.instagram.com 335
www.car-insurance-quote-fx.com 325
ma.brazzers.com 297
www.lipstickalley.com 288
www.earnhoney.com 273
www.yahoo.com 259
accounts.google.com 251
theadvocate.com 246
www.zulily.com 246
www.twitch.tv 241
buffer.com 241
en.wikipedia.org 241
news.google.com 232
apps.facebook.com 23

In [29]:
print_counter(overall_domains_where_reference_has_no_reconstructed)

www.mturk.com 196943
www.facebook.com 102365
vidzi.tv 32765
turkopticon.ucsd.edu 28882
www.mturkgrind.com 27821
www.mturkcrowd.com 26589
www.google.com 26488
www.reddit.com 25662
aurafc.guildwork.com 25227
www.nextmuni.com 23065
mail.google.com 22002
news.google.com 21932
us-mg5.mail.yahoo.com 20792
www.youtube.com 20307
my.presonus.com 19924
www.amazon.com 19431
www.netflix.com 19162
docs.python.org 18694
prolific.ac 14655
secure.bankofamerica.com 14078
www.hulu.com 11600
68.45.57.129:22362 10537
gate.aon.com 10441
community.babycenter.com 9353
www.txlottery.org 9172
perk.tv 8887
www.buzzfeed.com 8853
www.tumblr.com 8577
stonybrookuniversity.co1.qualtrics.com 8417
www.startribune.com 8103
armycivilianservice.usajobs.gov 7714
www.playstationtrophies.org 7628
sewg.az1.qualtrics.com 7284
www.toonova.com 6943
klbibkeccnjlkjkiokjodocebajanakg 6940
mintvine.com 6734
www.twitch.tv 6500
www.michigantrafficcrashfacts.org 6478
www.yahoo.com 6439
qbo.intuit.com 6438
www.kayak.com 6373
kat.cr 562

In [30]:
user_to_evaluation_stats = {}

for user in list_users():
  try:
    stats = get_evaluation_stats_for_user(user)
  except:
    continue
  user_to_evaluation_stats[user] = stats


In [31]:
overall_evaluation_stats = Counter()
for user,stats in user_to_evaluation_stats.items():
  for k,v in stats.items():
    overall_evaluation_stats[k] += v

print overall_evaluation_stats

Counter({'both_inactive': 141169725, 'ref_active_but_rec_inactive': 14179433, 'correct_url': 9480626, 'incorrect_domain': 2897043, 'ref_inactive_but_rec_active': 1157295, 'correct_domain': 1041077})


In [32]:
'''
def get_users_with_tab_focus_data():
  users_with_tab_focus_data = []

  for user in list_users():
    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):
      continue
    tab_focus_times = get_tab_focus_times_for_user(user)
    ordered_visits = get_history_ordered_visits_for_user(user)
    start_time = tab_focus_times[0]['start']
    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]
    #print len(tab_focus_times)
    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:
      users_with_tab_focus_data.append(user)
  return users_with_tab_focus_data
'''
#print len(users_with_tab_focus_data)

"\ndef get_users_with_tab_focus_data():\n  users_with_tab_focus_data = []\n\n  for user in list_users():\n    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):\n      continue\n    tab_focus_times = get_tab_focus_times_for_user(user)\n    ordered_visits = get_history_ordered_visits_for_user(user)\n    start_time = tab_focus_times[0]['start']\n    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]\n    #print len(tab_focus_times)\n    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:\n      users_with_tab_focus_data.append(user)\n  return users_with_tab_focus_data\n"

In [33]:
#user = 'dwCCD1uofp' # doing very poorly with this user
#user = 'UrZvGUsI8N'
#user = users_with_tab_focus_data[0]
#print user
#url_to_visits = get_history_visits_for_user(user)

In [34]:
'''
def compute_tab_focus_times_with_events_for_user(user):
  current_session_tracker = SessionTracker(store_events=True)
  for line in get_log_with_mlog_active_times_for_user(user):
    current_session_tracker.process_input(uncompress_data_subfields(line))
  current_session_tracker.end_input()
  return current_session_tracker.get_output()
'''

'\ndef compute_tab_focus_times_with_events_for_user(user):\n  current_session_tracker = SessionTracker(store_events=True)\n  for line in get_log_with_mlog_active_times_for_user(user):\n    current_session_tracker.process_input(uncompress_data_subfields(line))\n  current_session_tracker.end_input()\n  return current_session_tracker.get_output()\n'

In [35]:
#tab_focus_times_with_events = compute_tab_focus_times_with_events_for_user(user)


In [36]:
'''
for line in get_log_with_mlog_active_times_for_user(user):
  data = uncompress_data_subfields(line)
  url = get_focused_tab(data)
  if url != None:
    print url
    print data
    break
'''

'\nfor line in get_log_with_mlog_active_times_for_user(user):\n  data = uncompress_data_subfields(line)\n  url = get_focused_tab(data)\n  if url != None:\n    print url\n    print data\n    break\n'

In [37]:
#print len(tab_focus_times_with_events)
#tab_focus_times_with_events[0]

In [38]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print len(tab_focus_times)
#print tab_focus_times[0]
#print tab_focus_times

In [39]:
#ordered_visits = get_history_ordered_visits_for_user(user)
#first_site = url_to_visits.keys()[0]
#print url_to_visits[first_site]

In [40]:
'''
ordered_visits_after_start = []
for visit in ordered_visits:
  time = visit['visitTime']
  #if 1458171960840.0 <= time <= 1458172165207.0:
  #  print visit
  if time >= 1458171960840.0:
    ordered_visits_after_start.append(visit)
'''

"\nordered_visits_after_start = []\nfor visit in ordered_visits:\n  time = visit['visitTime']\n  #if 1458171960840.0 <= time <= 1458172165207.0:\n  #  print visit\n  if time >= 1458171960840.0:\n    ordered_visits_after_start.append(visit)\n"

In [41]:
#print len(ordered_visits_after_start)

In [42]:
'''
def to_visit_lengths(ordered_visits):
  focus_time_reconstructor = ReconstructFocusTimesBaseline()
  for visit in ordered_visits:
    focus_time_reconstructor.process_history_line(visit)
  return focus_time_reconstructor.get_output()

reconstructed_visit_lengths = to_visit_lengths(ordered_visits)
'''

'\ndef to_visit_lengths(ordered_visits):\n  focus_time_reconstructor = ReconstructFocusTimesBaseline()\n  for visit in ordered_visits:\n    focus_time_reconstructor.process_history_line(visit)\n  return focus_time_reconstructor.get_output()\n\nreconstructed_visit_lengths = to_visit_lengths(ordered_visits)\n'

In [43]:
#print len(reconstructed_visit_lengths)

In [44]:
#print ref_start_time

In [45]:
#print evaluated_tab_focus_times[0]

In [46]:
'''
for x in reconstructed_visit_lengths:
  url = x['url']
  if 'tmi.stanford.edu' in url:
    print x
    break
'''

"\nfor x in reconstructed_visit_lengths:\n  url = x['url']\n  if 'tmi.stanford.edu' in url:\n    print x\n    break\n"

In [47]:
#print len(evaluated_reconstructed_visits)
#print len(evaluated_tab_focus_times)

In [48]:
#def 

In [49]:
#print evaluated_reconstructed_visits[0]
#print evaluated_tab_focus_times[2]
#print evaluated_reconstructed_visits[-1]
#print evaluated_tab_focus_times[-1]

In [50]:
#print len(result_reference.get_output())
#print len(result_reconstructed.get_output())

In [51]:
'''
for ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:
  ref_domain = 'None'
  rec_domain = 'None'
  if ref_answer != 0:
    ref_url = result_reference.get_url_for_index(ref_answer)
    ref_domain = url_to_domain(ref_url)
  if rec_answer != 0:
    rec_url = result_reconstructed.get_url_for_index(rec_answer)
    rec_domain = url_to_domain(rec_url)
  print ref_domain, rec_domain
'''

"\nfor ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:\n  ref_domain = 'None'\n  rec_domain = 'None'\n  if ref_answer != 0:\n    ref_url = result_reference.get_url_for_index(ref_answer)\n    ref_domain = url_to_domain(ref_url)\n  if rec_answer != 0:\n    rec_url = result_reconstructed.get_url_for_index(rec_answer)\n    rec_domain = url_to_domain(rec_url)\n  print ref_domain, rec_domain\n"

In [52]:
#for x in evaluated_reconstructed_visits:
#  print x

In [53]:
#for x in evaluated_tab_focus_times:
#  print x

In [54]:
#for x in reconstructed_visit_lengths:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [55]:
#for x in tab_focus_times:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [56]:

'''
def to_visit_lengths(ordered_visits):
  output = []
  for item,idx in ordered_visits:
    url = item['url']
    visit_time = item['visitTime'] # timestamp in milliseconds
    result = {'url': url, 'start': visit_time, 'active': visit_time}
    visit_duration = 30*1000 # 30 seconds in milliseconds
    if idx < len(ordered_visits):
      next_item = ordered_visits[idx+1]
      next_visit_time = next_item['visitTime']
      time_until_next_visit = next_visit_time - visit_time
      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit
        
      visit_duration = min(visit_duration, time_until_next_visit)
    output.append({'url': url, 'start': visit_time, })
'''

"\ndef to_visit_lengths(ordered_visits):\n  output = []\n  for item,idx in ordered_visits:\n    url = item['url']\n    visit_time = item['visitTime'] # timestamp in milliseconds\n    result = {'url': url, 'start': visit_time, 'active': visit_time}\n    visit_duration = 30*1000 # 30 seconds in milliseconds\n    if idx < len(ordered_visits):\n      next_item = ordered_visits[idx+1]\n      next_visit_time = next_item['visitTime']\n      time_until_next_visit = next_visit_time - visit_time\n      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit\n        \n      visit_duration = min(visit_duration, time_until_next_visit)\n    output.append({'url': url, 'start': visit_time, })\n"